In [42]:
from datasets import Dataset
from src.utils import utils
from datasets import concatenate_datasets, load_from_disk
import json



In [43]:
preds = load_from_disk("/data/mourad/narratives/model_json_preds/gpt4o_proquest_v3")
preds = preds.to_pandas()

In [44]:
preds.completion = preds.completion.apply(lambda x: json.loads(x) if isinstance(x, str) else x)

In [45]:
def help(x):
    # print(x)
    narrs = utils.extract_narrative_category(x)
    if 'contains_narrative' in narrs:
        narrs.remove('contains_narrative')
    # if len(narrs) == 0:
    #     return None
    return narrs


In [46]:
preds['model'] = preds['completion'].apply(help)
preds = preds.dropna(subset=['model'])

In [47]:
gold_test = utils.load_hf_dataset(path=f"/data/mourad/narratives/sft_data", dataset='proquest')['test'].to_pandas()
# gold_df = pd.concat([gold_train, gold_test])
gold_df = gold_test
gold_df['annotation'] = gold_df.apply(lambda x: json.loads(utils.reconstruct_training_input(x)), axis=1)
gold_df['human'] = gold_df['annotation'].swifter.progress_bar(True).apply(help) #
# gold_df.narrative = gold_df.narrative.apply(lambda x: x)
gold_df

/data/mourad/narratives/sft_data_proquest


Pandas Apply: 100%|██████████| 488/488 [00:00<00:00, 445057.70it/s]


,id,text,assigned,template,data,annotation,narrative
0,59827,"In the first instance, accelerating inflation ...",az*,"{\n""foreign"": #,\n""contains-narrative"": #,\n""i...","false#true#present#""up""#[{""effect"": ""savings"",...","{'foreign': False, 'contains-narrative': True,...",[savings]
1,59945,"she said,""but the old folks are still caught i...",az*,"{\n""foreign"": #,\n""contains-narrative"": #,\n""i...","false#true#present#""up""#[{""effect"": ""cost"", ""t...","{'foreign': False, 'contains-narrative': True,...",[cost]
2,59988,"""Congress' action was part of a general respon...",az*,"{\n""foreign"": #,\n""contains-narrative"": #,\n""i...","false#true#present#""na""#[{""effect"": ""govt"", ""t...","{'foreign': False, 'contains-narrative': True,...",[govt]
3,59790,The White House Council on Wage and Price Stab...,az*,"{\n""foreign"": #,\n""contains-narrative"": #,\n""i...",false#false,"{'foreign': False, 'contains-narrative': False...",[]
4,59953,DRUM CORPS NOTES Inflation woes hit contest sp...,az*,"{\n""foreign"": #,\n""contains-narrative"": #,\n""i...",false#false,"{'foreign': False, 'contains-narrative': False...",[]
...,...,...,...,...,...,...,...
483,59785,The only honest way to judge the success of Ph...,az*,"{\n""foreign"": #,\n""contains-narrative"": #,\n""i...",false#false,"{'foreign': False, 'contains-narrative': False...",[]
484,59939,"inflation.""",az*,"{\n""foreign"": #,\n""contains-narrative"": #,\n""i...",false#false,"{'foreign': False, 'contains-narrative': False...",[]
485,59615,"""We should have a list In two weeks of things ...",az*,"{\n""foreign"": #,\n""contains-narrative"": #,\n""i...",false#false,"{'foreign': False, 'contains-narrative': False...",[]
486,59703,Adamantly refusing to increase Kunguru retail ...,az*,"{\n""foreign"": #,\n""contains-narrative"": #,\n""i...","false#true#present#""up""#[{""effect"": ""cost"", ""t...","{'foreign': False, 'contains-narrative': True,...",[cost]


In [48]:
gold_df = gold_df[["text", "narrative"]]
preds = preds[["text", "completion"]]
df = gold_df.merge(preds, on='text', how='inner')

In [50]:
diff_df = df[df.narrative != df.completion]



diff_df.to_csv("../../data/for_labeling/gpt4o_vs_human_proquest_v3.csv", index=False)